In [ ]:
from cnn_method import *
from cnn_preproc_function import *

import optparse
import numpy as np
import pandas as pd

import sklearn
from sklearn import preprocessing

import keras
import tensorflow as tf

from keras_model_configuration import *
from keras_metric import *

import datetime
import time
import os

from sklearn.externals import joblib

#OPTS PARSER
# optparser = optparse.OptionParser()
# optparser.add_option("-a", "--assetindex", default=0, help="assetindex")
# optparser.add_option("-d", "--gpudevice", default="0", help="gpudevice")
# opts = optparser.parse_args()[0]

# assetindex = int(opts.assetindex)
# gpudevice = opts.gpudevice
asset_index=0
gpudevice="0"

#PARAMS:
loss_criteria = "precision"
loss_mode = "max"

#GPU CONFIG
os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"]=gpudevice
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)


#SETUP INIT
setup_folder_name = "AUDUSD_1540177760"
setup_path = os.path.join("output", setup_folder_name)

DATA_PARAMS = joblib.load(os.path.join(setup_path, "DATA_PARAMS.pkl"))
MODEL_PARAMS = joblib.load(os.path.join(setup_path, "MODEL_PARAMS.pkl"))
scaler = joblib.load(os.path.join(setup_path, "scaler.pkl"))

locals().update(DATA_PARAMS)
locals().update(MODEL_PARAMS)

print(DATA_PARAMS)
print(MODEL_PARAMS)

df = load_data(raw_data_file)
df = clean_and_create_target(df, TARGET_TO_PREDICT, FUTURE_PERIOD_PREDICT, TARGET_FUNCTION, TARGET_THRESHOLD, FLIP)
df, timestamp, all_data_gen = FullTSGenerator(df, scaler, 512, SEQ_LEN, old = True)


#pick best model
models_folder = os.path.join(setup_path, "models")
files = [f for f in os.listdir(models_folder) if os.path.isfile(os.path.join(models_folder,f))]
metric_dict = dict()
metrics_1 = ["index", "loss", "accuracy", "precision", "f1"]
for j,v in enumerate(metrics_1):
    try:
        metric_dict[v] = [int(f.split("-")[j+1]) for f in files]
    except:
        metric_dict[v] = [np.nan for f in files]

if loss_mode == "max":
    best_index = np.argmax(metric_dict[loss_criteria])
else:
    best_index = np.argmin(metric_dict[loss_criteria])

best_model_file = files[best_index]
best_model_path = os.path.join(models_folder,best_model_file)


print("Best Model: Done!")

In [ ]:
model = keras.models.load_model(best_model_path,
            custom_objects=None,
            compile=False
        )
# adm = keras.optimizers.Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False, decay = 1e-6)
# model.compile(optimizer=adm, loss='binary_crossentropy', metrics=['accuracy', precision, f1])

# print(y)
# print(np.array(y).shape)


In [ ]:
y2 = model.predict_generator(all_data_gen, steps = 100)



In [ ]:
g = True
"asd_" + g

In [ ]:
df = pd.DataFrame(dict(Date = timestamp, raw_signal = y.flatten()))

In [ ]:
df.set_index("Date").plot()